In [53]:
# take data 
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

url = 'https://voz.vn/t/event-box-cntt-2023-chia-se-kinh-nghiem-phong-van.694369/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'
}

response = urlopen(Request(url=url, headers=headers))

if response.getcode() == 200:
    soup = BeautifulSoup(response, 'html.parser')
    #DataUserId = 
    print(response.getcode())
    #DataUserName = scrapy.Field()
    #ReplyTime = scrapy.Field()
    #ReplyText = scrapy.Field()
    #QuoteBlockId = scrapy.Field()
    #QuoteBlockText = scrapy.Field()
    #print(soup.prettify())  # Hiển thị HTML đã lấy được
else:
    print(f"Lỗi khi lấy trang: {response.getcode()}")


200


In [2]:
dict_user = {}
articles = soup.find_all(
    "article", class_="message message--post js-post js-inlineModContainer"
                       
)
# Duyệt qua từng article và lấy thông tin
for article in articles:
    # Tìm thẻ con message-userDetails
    user_details = article.find("div", class_="message-userDetails")

    if user_details:
        # Tìm thẻ con chứa message-name
        message_name = user_details.find("a", class_="username")

        if message_name:
            # Lấy giá trị data-user-id và text
            user_id = message_name.get("data-user-id")  # Lấy thuộc tính data-user-id
            user_name = message_name.text  # Lấy văn bản trong thẻ
            dict_user[user_id] = user_name
print(dict_user)

{'873787': 'Fire Of Heart', '1653529': 'teeeeeeeee', '1719893': 'minhducitus', '1781147': 'midnight_sun', '1418575': 'luiz', '1797102': 'NatriClorid-0.9%', '1180502': 'J11_PRO', '1669459': 'JuniorDeveloper', '1631237': 'YG Ren', '1111831': 'coldkaldr', '1373107': 'gacononline1', '1673518': 'NextLesorNextVoz', '115818': 'tdat00'}


In [49]:

import re
for article in articles:
    # Tìm thẻ con message-userDetails
    user_details_content = article.find('div', class_='message-userContent lbContainer js-lbContainer')
    #print({"a":user_details.get("data-lb-caption-desc"),
    #       "b": user_details.get("data-lb-id")})               
    #block_quote = article.find('div', class_='message-userContent lbContainer js-lbContainer')
    if user_details_content:
        # Tìm thẻ con chứa message-name
        message_id = re.findall(r'\d+',user_details_content.get("data-lb-id"))[0]
        message_time = parser_time(user_details_content.get("data-lb-caption-desc"))
       
        #print(message_name)
    if user_details_content :
        blockquote = user_details_content.find('article', class_='message-body js-selectToQuote')
        
        data = {}
        blockquote1 = user_details_content.find_all('blockquote', class_='bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch')
        ad_list = []
        for block in blockquote1: # one person can quote more block
                  block_id = re.findall(r'\d+',block.get("data-attributes"))
                  data_source = re.findall(r'\d+',block.get("data-source"))
                  data_text = block.get_text()
                  ad_list.append(data_source)
                  ad_list.append({"blockid":block_id[0],
                         "data_source":data_source[0],
                         "data_text":data_text})
        print({"message_id": message_id,
               "message_time" : message_time,
               "quote" : ad_list,
              "text":blockquote.getText(),
              })
                  
       

  

{'message_id': '16589316', 'message_time': datetime.datetime(2022, 3, 23, 9, 47), 'quote': [], 'text': '\n\nThread cũ bị google quét. Xem lại nội dung cũ ở đây.\nhttps://web.archive.org/web/2022032...-chia-se-ve-muc-luong-tai-cac-cong-ty.459731/\n\n\nNội quy thread mới:\n\nChỉ comment nội dung như sau:\n\nTên công ty: (bắt buộc, review ko có tên cty thì nghỉ đi)\nLương tháng/năm (gross):\nVị trí:\nThời điểm (năm nào):\nBonus: (Tháng 13-14-15/ thưởng performance/thưởng lễ tết/stock...)\nSố năm kinh nghiệm khi nhận offer:\nNên ghi thêm các phúc lợi, mức đóng thuế, stock, bảo hiểm, số năm kinh nghiệm ....\n\n\nCác nội dung comment không liên quan sẽ bị xoá và warn.\n\n\xa0\n'}
{'message_id': '16589910', 'message_time': datetime.datetime(2022, 3, 23, 9, 47), 'quote': [], 'text': '\n\nThread cũ. https://voz.vn/t/thread-tong-hop-chia-se-ve-muc-luong-tai-cac-cong-ty.459731/ có 45 page. Cần khoảng 11 bạn, mỗi bạn nhận khoảng 4 page, bắt đầu từ page 2.\nCông việc như sau:\n\nVào trang https://w

In [41]:
from dateutil import parser

# Chuỗi ngày giờ cần phân tích
date_string = "Fire Of Heart · Mar 23, 2022 at 9:47 AM"

# Phân tích và chuyển đổi chuỗi ngày giờ
# Lấy phần ngày giờ từ chuỗi, ví dụ sử dụng biểu thức chính quy để trích xuất phần ngày giờ
import re

# Biểu thức chính quy để tìm phần ngày giờ
date_match = re.search(r'\b\w{3} \d{2}, \d{4} at \d{1,2}:\d{2} (AM|PM)\b', date_string)
if date_match:
    date_str = date_match.group(0)
    # Phân tích chuỗi ngày giờ thành đối tượng datetime
    date = parser.parse(date_str)
    print(date)
else:
    print("No date found in the string")


2022-03-23 09:47:00


In [43]:
def parser_time(string):
    # Phân tích và chuyển đổi chuỗi ngày giờ
    # Lấy phần ngày giờ từ chuỗi, ví dụ sử dụng biểu thức chính quy để trích xuất phần ngày giờ
    date_match = re.search(
        r"\b\w{3} \d{2}, \d{4} at \d{1,2}:\d{2} (AM|PM)\b", date_string
    )

    if date_match:
        date_str = date_match.group(0)
        # Phân tích chuỗi ngày giờ thành đối tượng datetime
        date = parser.parse(date_str)
        return date
    else:
        return "No date found in the string"
parser_time("Fire Of Heart · Mar 23, 2022 at 9:47 AM")

datetime.datetime(2022, 3, 23, 9, 47)

In [51]:
import requests

session = requests.Session()
url = 'https://voz.vn/t/event-box-cntt-2023-chia-se-kinh-nghiem-phong-van.694369/'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}
response = session.get(url, headers=headers)
print(response.status_code)

403
